In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [29]:
from langchain.chains.summarize import load_summarize_chain
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import pipeline
import torch
import base64

c:\Users\novil\anaconda3\envs\summy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
checkpoint = "./LaMini-Flan-T5-248M"
tokenizer = T5Tokenizer.from_pretrained(checkpoint )

base_model = T5ForConditionalGeneration.from_pretrained(checkpoint , device_map='auto',
                                                        torch_dtype = torch.float32,
                                                        offload_folder = './LaMini-Flan-T5-248M')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")
# model = AutoModelForSeq2SeqLM.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")

In [35]:
pipe = pipeline(
    "summarization",
    model = base_model,
    tokenizer=tokenizer,
    max_length = 500,
    min_length = 50
)
def summy(input_text, pipe = pipe):
    result = pipe(input_text)
    summary = result[0]['summary_text']

    return summary

In [21]:
text = """Semantic search seeks to improve search accuracy by understanding the content of the search query instead of relying on lexical matching only. This is done leveraging similarities between embeddings.

The idea behind semantic search is to embed all entries in your corpus into a vector space. At search time, the query is embedded into the same vector space and the closest embeddings from your corpus are found.

Semantic Search can be performed using the semantic_search function of the util module, which works on the embeddings of the documents in a corpus and on the embeddings of the queries.
"""

In [37]:
summy(text)

Your max_length is set to 500, but your input_length is only 137. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)


'Semantic search improves search accuracy by understanding the content of the search query by leveraging similarities between embeddings. It embeds all entries in a corpus into a vector space at search time, and at the same time, the closest embeddas from the corpus are found. The semantic_search function of the util module works on the embedddings of the documents and the queries.'

In [38]:
"""
'Semantic search improves search accuracy by understanding the content of the search query by leveraging similarities between embeddings. It embeds all entries in a corpus into a vector space at search time, and at the same time, the closest embeddas from the corpus are found. The semantic_search function of the util module works on the embedddings of the documents and the queries.'

"""

"\n'Semantic search improves search accuracy by understanding the content of the search query by leveraging similarities between embeddings. It embeds all entries in a corpus into a vector space at search time, and at the same time, the closest embeddas from the corpus are found. The semantic_search function of the util module works on the embedddings of the documents and the queries.'\n\n"

#### New pipe line

In [41]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


checkpoint = "./LaMini-Flan-T5-248M"
tokenizer = T5Tokenizer.from_pretrained(checkpoint )

base_model = T5ForConditionalGeneration.from_pretrained(checkpoint , device_map='auto', torch_dtype = torch.float32, offload_folder="offload")

pipe = pipeline(
    "summarization",
    model = base_model,
    tokenizer=tokenizer,
    max_length = 500,
    min_length = 50
)

hf = HuggingFacePipeline(pipeline=pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
from langchain_core.prompts import PromptTemplate

template =""" give a title for the given input text.

Input text : {input_text}

return the answer in the following format.

Answer: 
"""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf



In [43]:
input_text = """In this section we compare various aspects of self-attention layers to the recurrent and convolutional layers commonly used for mapping one variable-length sequence of symbol representations
(x1, ..., xn) to another sequence of equal length (z1, ..., zn), with xi
, zi ∈ R
d
, such as a hidden
layer in a typical sequence transduction encoder or decoder. Motivating our use of self-attention we
consider three desiderata.
One is the total computational complexity per layer. Another is the amount of computation that can
be parallelized, as measured by the minimum number of sequential operations required.
The third is the path length between long-range dependencies in the network. Learning long-range
dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the
ability to learn such dependencies is the length of the paths forward and backward signals have to
traverse in the network. The shorter these paths between any combination of positions in the input
and output sequences, the easier it is to learn long-range dependencies [12]. Hence we also compare
the maximum path length between any two input and output positions in networks composed of the
different layer types.
As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially
executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of
computational complexity, self-attention layers are faster than recurrent layers when the sequence length n is smaller than the representation dimensionality d, which is most often the case with
sentence representations used by state-of-the-art models in machine translations, such as word-piece
[38] and byte-pair [31] representations. To improve computational performance for tasks involving
very long sequences, self-attention could be restricted to considering only a neighborhood of size r in
the input sequence centered around the respective output position. This would increase the maximum
path length to O(n/r). We plan to investigate this approach further in future work"""

In [44]:

print(chain.invoke({"input_text": input_text}))

Your max_length is set to 500, but your input_length is only 477. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=238)


Title: Comparing Self-attention Layers to Recurrent and Convolutional Layers for Sequence Transduction Tasks and Learning Long-Range Dependencies in Networks with Xi, Zi  R d and Path Length between Long-Response Signals.


In [ ]:
"""The text compares self-attention layers to recurrent and convolutional layers for mapping variable-length sequences of symbol representations and compares the computational complexity per layer, parallelization, and path length between long-range dependencies in the network. The author explains that learning long-rang dependencies is a key challenge in many sequence transduction tasks and considers the length of the paths forward and backward signals to improve computational performance. Self-attention is faster when the sequence length n is smaller than the representation dimensionality d, which is most often used with sentence representations used by state-of-the-art models in machine translations.


Title: Comparing Self-attention Layers to Recurrent and Convolutional Layers for Sequence Transduction Tasks and Learning Long-Range Dependencies in Networks with Xi, Zi  R d and Path Length between Long-Response Signals.



"""

#### Testing Vector DB

In [2]:
from langchain_chroma import Chroma

In [3]:
from langchain_core.documents import Document

In [8]:
import uuid 
  
id = uuid.uuid1(1) 

In [9]:
id.int

280304693971884119651480603231246090241

In [17]:
uuid.uuid1().int, uuid.uuid4().int

(110661751864737925840623571635704135056,
 21554683291254377879421471393988273732)

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [26]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.split_text(text)


In [27]:
len(docs)

8

In [28]:
docs

['Semantic search seeks to improve search accuracy by understanding the content of the search query',
 'query instead of relying on lexical matching only. This is done leveraging similarities between',
 'between embeddings.',
 'The idea behind semantic search is to embed all entries in your corpus into a vector space. At',
 'space. At search time, the query is embedded into the same vector space and the closest embeddings',
 'from your corpus are found.',
 'Semantic Search can be performed using the semantic_search function of the util module, which works',
 'works on the embeddings of the documents in a corpus and on the embeddings of the queries.']

#### testing mysql connector

In [2]:
import sqlite3


In [3]:

# Creating a connection object to a database file
conn = sqlite3.connect('example.db')
# Creating a cursor object to execute SQL commands
cur = conn.cursor()
# Creating a table named users with two columns: id and name
cur.execute('CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT)')
# Inserting some records into the table
cur.execute('INSERT INTO users (name) VALUES ("Alice")')
cur.execute('INSERT INTO users (name) VALUES ("Bob")')
cur.execute('INSERT INTO users (name) VALUES ("Don")')
# Committing the changes to the database
conn.commit()
# Querying the table and fetching all the records
cur.execute('SELECT * FROM users')
rows = cur.fetchall()
# Printing the records
for row in rows:
    print(row)
# Closing the connection
conn.close()


(1, 'Alice')
(2, 'Bob')
(3, 'Don')


In [5]:
conn = sqlite3.connect('example.db')
# Creating a cursor object to execute SQL commands
cur = conn.cursor()
name = "a@#b"
cur.execute(f'''Select * from users where name = "{name}"''')
records = cur.fetchall()
records

[]

In [4]:
f'Select * from users where name = "{name}"'

'Select * from users where name = "a@#b"'

#### QA pipeline

In [1]:
# from langchain.document_loaders import HuggingFaceDatasetLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

c:\Users\novil\anaconda3\envs\summy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# prompt_template = """Use the following pieces of context to answer the question at the end. If the answer can be inferred from the context, provide it with proper punctuation and capitalization. 
# If the answer is not present in the context but can be addressed with your model knowledge up to your last update, use that information to answer. 
# If you still don't know the answer, state that you don't know. Do not fabricate an answer.

# {context}

# Question: {question}
# Generate point-wise answers for the question if required, and format them clearly with bullet points or numbers.
# Conclude or explain the context based on the question's intent when possible.
# If the answer is not present in the context, attempt to provide a close enough answer using the model's knowledge without making assumptions beyond that knowledge.
# Answer:"""
# PROMPT = PromptTemplate(
#     template=prompt_template, input_variables=["context", "question"]#, "sources"]
# )

In [2]:
# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"
# model_name = "./LaMini-Flan-T5-248M"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering", 
    model=model_name, 
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

In [3]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
# retriever = db.as_retriever(search_kwargs={"k": 4})

# # Create a question-answering instance (qa) using the RetrievalQA class.
# # It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [8]:
input_text = """In this section we compare various aspects of self-attention layers to the recurrent and convolutional layers commonly used for mapping one variable-length sequence of symbol representations
(x1, ..., xn) to another sequence of equal length (z1, ..., zn), with xi
, zi ∈ R
d
, such as a hidden
layer in a typical sequence transduction encoder or decoder. Motivating our use of self-attention we
consider three desiderata.
One is the total computational complexity per layer. Another is the amount of computation that can
be parallelized, as measured by the minimum number of sequential operations required.
The third is the path length between long-range dependencies in the network. Learning long-range
dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the
ability to learn such dependencies is the length of the paths forward and backward signals have to
traverse in the network. The shorter these paths between any combination of positions in the input
and output sequences, the easier it is to learn long-range dependencies [12]. Hence we also compare
the maximum path length between any two input and output positions in networks composed of the
different layer types.
As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially
executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of
computational complexity, self-attention layers are faster than recurrent layers when the sequence length n is smaller than the representation dimensionality d, which is most often the case with
sentence representations used by state-of-the-art models in machine translations, such as word-piece
[38] and byte-pair [31] representations. To improve computational performance for tasks involving
very long sequences, self-attention could be restricted to considering only a neighborhood of size r in
the input sequence centered around the respective output position. This would increase the maximum
path length to O(n/r). We plan to investigate this approach further in future work"""

In [11]:
from langchain_core.documents import Document
ld = Document(page_content=input_text)

In [15]:
type(ld)

langchain_core.documents.base.Document

In [16]:
# llm_model = OpenAIChat(temperature=0.25,model_name="gpt-3.5-turbo-1106")

template = """ summarize the given text 
text : {context}
"""
PROMPT = PromptTemplate(
    template=template, input_variables=["context"]#, "sources"]
)

chain=load_qa_chain(llm,  chain_type="stuff", prompt=PROMPT)
res=chain({"input_documents": ld, "context": input_text })


AttributeError: 'tuple' object has no attribute 'page_content'

#### API testing

In [7]:
import requests
import json

url = " http://127.0.0.1:5000"

In [8]:
email = 'novil@gmail.com'
pwd = 'N0M@1001'
res = requests.get(url+'/login', json={'email':email, 'password':pwd})

In [9]:
res

<Response [200]>

In [10]:
res.json()

{'log': 'No Details Found'}

In [28]:
full_name = 'Novil sai'


res = requests.get(url+'/update_user', json={'email':email, 'password':pwd, "full_name":full_name})

In [29]:
res

<Response [200]>

In [30]:
res.json()

{'log': 'Exists'}

In [31]:
text = """RAG, or retrieval-augmented generation, is a new way to understand and create language. It combines two kinds of models. First, retrieve relevant information. Second, generate text from that information. By using both together, RAG does an amazing job. Each model’s strengths make up for the other’s weaknesses. So RAG stands out as a groundbreaking method in natural language processing.
RAG combines retrieval models that extract data from vast knowledge repositories with generative models that formulate pertinent responses. RAG produces responses rooted in factual information.
The interplay between these components enables RAG to be more informative and accurate than conventional generational models operating independently.
RAG has the skills to comprehend and answer hard questions. How does it work? It uses knowledge from searching to understand the content better, so its outputs are more relevant and informative.
RAG adapts well, easily working with different NLP apps like answering questions, dialogue systems, and making content. Its flexibility allows developers to use its abilities across many natural language understanding and generation tasks without much trouble.
Improved Accuracy: RAG combines the benefits of retrieval-based and generative models, leading to more accurate and contextually relevant responses.
Enhanced Contextual Understanding: By retrieving and incorporating relevant knowledge from a knowledge base, RAG demonstrates a deeper understanding of queries, resulting in more precise answers.
Reduced Bias and Misinformation: RAG’s reliance on verified knowledge sources helps mitigate bias and reduces the spread of misinformation compared to purely generative models.
Versatility: RAG can be applied to various natural language processing tasks, such as question answering, chatbots, and content generation, making it a versatile tool for language-related applications.
Empowering Human-AI Collaboration: RAG can assist humans by providing valuable insights and information, enhancing collaboration between humans and AI systems.
Advancement in AI Research: RAG represents a significant advancement in AI research by combining retrieval and generation techniques, pushing the boundaries of natural language understanding and generation.
"""

In [34]:
res = requests.get(url+"/summarize", json={'text':text, "email":'novil@gmail.com'})


In [36]:
res.json()

{'summary': 'RAG is a new method in natural language processing that combines two types of models: retrieval and generation. It combines retrieval models that extract relevant information and generative models that formulate pertinent responses. RAG has the skills to comprehend and answer hard questions. It works by using knowledge from searching to understand the content better, resulting in more relevant and informative outputs. It adapts well to different NLP apps and can be used across many natural language understanding and generation tasks without much trouble. Its reliance on verified knowledge sources helps mitigate bias and reduces the spread of misinformation compared to purely generative model. It can assist humans by providing valuable insights and information, enhancing collaboration between humans and AI systems. Advancement in AI research.'}

In [ ]:
"""{'summary': 'RAG is a new method in natural language processing that combines two types of models: retrieval and generation. It combines retrieval models that extract relevant information and generative models that formulate pertinent responses. RAG has the skills to comprehend and answer hard questions. It works by using knowledge from searching to understand the content better, resulting in more relevant and informative outputs. It adapts well to different NLP apps and can be used across many natural language understanding and generation tasks without much trouble. Its reliance on verified knowledge sources helps mitigate bias and reduces the spread of misinformation compared to purely generative model. It can assist humans by providing valuable insights and information, enhancing collaboration between humans and AI systems. Advancement in AI research.'}"""